# Seminar 6. Web applications with Python. Wrapping C/C++ code for Python.

## 6.1. Python for web applications

Some frameworks:
1. Flask
2. Django
3. CherryPy
4. bottlepy

Template Engines:
1. Jinja2
2. Mako

### 6.1.1. Template engine jinja2

Jinja2 is a full featured template engine with full unicode support. 

Installing:

    conda install jinja2
    pip install jinja2

Doc: http://jinja.pocoo.org/docs/dev/

In [1]:
%%writefile base.html

<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> {{ title }} </title>
  </head>
  <body>
{% block body %}
    Some text
{% endblock %}
  </body>
</html>

Overwriting base.html


In [2]:
import jinja2

j2env = jinja2.Environment(loader=jinja2.FileSystemLoader('.'), trim_blocks=True)
print(j2env.get_template('base.html').render(title='Blog name'))


<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> Blog name </title>
  </head>
  <body>
    Some text
  </body>
</html>


In [3]:
%%writefile body.html
{% extends "base.html" %}

{% block body %}
    {{ super() }}
    <ol>
    {% for item in items %}
      <li>{{ item }}</li>
    {% endfor %}
    </ol>
{% endblock %}

Overwriting body.html


In [4]:
print(j2env.get_template('body.html').render(title='Blog name', items=['apple', 'peach', 'pineapple']))


<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> Blog name </title>
  </head>
  <body>
        Some text

    <ol>
          <li>apple</li>
          <li>peach</li>
          <li>pineapple</li>
        </ol>
  </body>
</html>


### 6.1.2. bottle.py

Bottle is a fast, simple and lightweight single-file micro web-framework for Python with no dependencies other than the Python Standard Library. Bottle is customizable with plugins.

Installation:

    pip install bottle

Doc: https://bottlepy.org/

Examples below are from the official documentation.

In [5]:
%%writefile bottle_example.py
#!/usr/bin/env python
import os
import bottle

indexTemplate = 'Hello, <strong>{{ username }}</strong>.'

@bottle.route('/name/<name>')
def name(name):
    return bottle.template(indexTemplate, username=name)

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 9999))
    bottle.run(host='0.0.0.0', port=port, debug=True)

Overwriting bottle_example.py


In [6]:
%%bash
chmod +x bottle_example.py
./bottle_example.py

Process is interrupted.


Now on page http://localhost:9999/name/test you will get message «Hello, **test**.»

* Static routes (example: `@bottle.route('/')`)
* Dynamic routes with wildcards (example: `@bottle.route('/name/<name>')`)

Filters can be used to define more specific wildcards:

``` python
@bottle.route('/name/<id:int>')
def callback(id):
    return id

@bottle.route('/name/<name:path>')
def callback(name):
    return name
```

Full set of filters: int, float, path, re.

In [7]:
import bottle
@bottle.get('/login') # or @bottle.route('/login')
def login():
    return '''
        <form action="/login" method="post">
            Username: <input name="username" type="text" />
            Password: <input name="password" type="password" />
            <input value="Login" type="submit" />
        </form>
    '''

@bottle.post('/login') # or @bottle.route('/login', method='POST')
def do_login():
    username = bottle.request.forms.get('username')
    password = bottle.request.forms.get('password')
    if check_login(username, password):
        return "<p>Your login information was correct.</p>"
    else:
        return "<p>Login failed.</p>"

## 6.1.3. CherryPy module

CherryPy is a pythonic, object-oriented web framework.

Installation:
    
    conda install cherrypy
    pip install cherrypy
    
Doc: http://docs.cherrypy.org/en/latest/index.html

In [8]:
%%writefile base.html
<!DOCTYPE html>
<html>
  <head>
    <meta charset="utf-8">
      <title> {{ title }} </title>
  </head>
  <body>
    {% block body %}Some text{% endblock %}
  </body>
</html>

Overwriting base.html


In [9]:
%%writefile body.html

{% extends "base.html" %}

{% block body %}
    {{ super() }}
    <ol>
    {% for item in items %}
      <li>{{ item }}</li>
    {% endfor %}
    </ol>
{% endblock %}

Overwriting body.html


In [10]:
%%writefile app.py
#!/usr/bin/env python3
import cherrypy
import jinja2
import os

class Application():
    def __init__(self, blog_name):
        self.blog_name = blog_name
        current_dir = os.path.dirname(os.path.abspath(__file__))
        self.j2env = jinja2.Environment(loader=jinja2.FileSystemLoader(current_dir), trim_blocks=True)
    
    @cherrypy.expose
    def index(self):
        return self.j2env.get_template('base.html').render(title=self.blog_name)
    
    @cherrypy.expose
    def make_list(self, items):
        encoded_items = (x.encode('ISO-8859-1').decode('utf-8') for x in items.split(','))
        return self.j2env.get_template('body.html').render(title=self.blog_name, items=encoded_items)

if __name__ == '__main__':
   cherrypy.quickstart(Application('Test blog'), '/')


Overwriting app.py


In [11]:
%%bash
chmod +x app.py
./app.py

Process is interrupted.


On page http://127.0.0.1:8080/make_list/apple,road,cake you will see:

Some text
1. apple
2. road
3. cake

### 6.1.4 CherryPy + Ajax

Example from the official documentation: http://docs.cherrypy.org/en/latest/tutorials.html#tutorial-8-make-it-smoother-with-ajax

In [12]:
%%writefile index.html
<!DOCTYPE html>
<html>
   <head>
     <script src="http://code.jquery.com/jquery-2.0.3.min.js"></script>
     <script type="text/javascript">
       $(document).ready(function() {

         $("#generate-string").click(function(e) {
           $.post("/generator", {"length": $("input[name='length']").val()})
            .done(function(string) {
               $("#the-string").show();
               $("#the-string input").val(string);
            });
           e.preventDefault();
         });

         $("#replace-string").click(function(e) {
           $.ajax({
              type: "PUT",
              url: "/generator",
              data: {"another_string": $("#the-string input").val()}
           })
           e.preventDefault();
         });

         $("#delete-string").click(function(e) {
           $.ajax({
              type: "DELETE",
              url: "/generator"
           })
           .done(function() {
              $("#the-string").hide();
           });
           e.preventDefault();
         });
                
         $.ajax({
           type: "GET",
           url: "/generator"
         }).done(function(string) {
            $("#the-string input").val(string);
        }
         );
       });
     </script>
   </head>
   <body>
     <input type="text" value="20" name="length" />
     <button id="generate-string">Generate string</button>
     <div id="the-string">
         <input type="text" />
         <button id="replace-string">Replace</button>
         <button id="delete-string">Delete it</button>
     </div>
   </body>
</html>

Overwriting index.html


In [13]:
%%writefile tut8.py
#!/usr/bin/env python3
import os, os.path
import random
import string

import cherrypy

class StringGenerator(object):
   @cherrypy.expose
   def index(self):
       return open('index.html')

class StringGeneratorWebService(object):
    exposed = True

    @cherrypy.tools.accept(media='text/plain')
    def GET(self):
        return cherrypy.session['mystring']

    def POST(self, length=8):
        some_string = ''.join(random.sample(string.hexdigits, int(length)))
        cherrypy.session['mystring'] = some_string
        return some_string

    def PUT(self, another_string):
        cherrypy.session['mystring'] = another_string

    def DELETE(self):
        cherrypy.session.pop('mystring', None)

if __name__ == '__main__':
    conf = {
        '/': {
            'tools.sessions.on': True,
            'tools.staticdir.root': os.path.abspath(os.getcwd())
        },
        '/generator': {
            'request.dispatch': cherrypy.dispatch.MethodDispatcher(),
            'tools.response_headers.on': True,
            'tools.response_headers.headers': [('Content-Type', 'text/plain')],
        },
    }
    webapp = StringGenerator()
    webapp.generator = StringGeneratorWebService()
    cherrypy.quickstart(webapp, '/', conf)

Overwriting tut8.py


In [14]:
%%bash
chmod +x tut8.py
./tut8.py

Process is interrupted.


Open page: On page http://127.0.0.1:8080/

## 6.2. C++ with python

### 6.2.1. swig

http://www.swig.org/

SWIG — Simplified Wrapper and Interface Generator. It supports many languages (C/C++ to that language), among them:

* javascript
* python
* go
* java
* etc

Installation:

    sudo apt-get install swig

In [15]:
%%writefile swtest.cpp

#include <omp.h>

extern "C" {
    void MarkBoundaries(unsigned int *raw, int N) {
        #pragma omp parallel for
        for (int x = 0; x < N; ++x) {
            raw[x] = 3;
        }
    }
}

Overwriting swtest.cpp


In [16]:
%%bash
wget https://raw.githubusercontent.com/numpy/numpy/master/tools/swig/numpy.i -O numpy.i

--2017-03-24 06:10:02--  https://raw.githubusercontent.com/numpy/numpy/master/tools/swig/numpy.i
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.12.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108861 (106K) [text/plain]
Saving to: ‘numpy.i’

     0K .......... .......... .......... .......... .......... 47%  554K 0s
    50K .......... .......... .......... .......... .......... 94% 1,57M 0s
   100K ......                                                100% 45,7M=0,1s

2017-03-24 06:10:02 (876 KB/s) - ‘numpy.i’ saved [108861/108861]



In [17]:
%%writefile swtest.swig

%module swtest
%{
    #define SWIG_FILE_WITH_INIT
    #include <omp.h>
    extern "C" {
        void MarkBoundaries(unsigned int* raw, int N);
    }
%}

%include "numpy.i"
%init %{
    import_array();
%}

%apply (unsigned int* IN_ARRAY1, int DIM1) {(unsigned int* raw, int N)};

#include <omp.h>
extern "C" {  
    extern void MarkBoundaries(unsigned int* raw, int N);
}

Overwriting swtest.swig


In [28]:
%%bash
swig -python swtest.swig
g++ -fopenmp -fPIC -c swtest.cpp swtest_wrap.c -I/usr/include/python3.4
g++ -fopenmp -shared swtest.o swtest_wrap.o -o _swtest.so

In [29]:
import numpy
import swtest

raw = numpy.ones(125, dtype=numpy.uint32)
raw[10:35] = numpy.zeros(25, dtype=numpy.uint32)

print(raw)

swtest.MarkBoundaries(raw)

print(raw)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


### 6.2.2. cffi

In [20]:
%%writefile cftest.cpp
#include <omp.h>

extern "C" {
    void MarkBoundaries(unsigned int *raw, int N) {
        #pragma omp parallel for
        for (int x = 0; x < N; ++x) {
            raw[x] = 3;
        }
    }
}

Overwriting cftest.cpp


In [21]:
%%bash
g++ -fPIC -c cftest.cpp -fopenmp -O3
g++ -fopenmp -shared cftest.o -o _cftest.so

In [22]:
%%writefile cftest.py

from cffi import FFI
import os
_ffi = FFI()
_lib = _ffi.dlopen(os.path.join(os.path.dirname(os.path.realpath(__file__)), "_cftest.so"))
_ffi.cdef("""
void MarkBoundaries(unsigned int *raw, int N);
""")

def MarkBoundaries(raw):
    dataPtr = _ffi.cast("unsigned int *", raw.ctypes.data)
    _lib.MarkBoundaries(dataPtr, raw.size)

Overwriting cftest.py


In [23]:
import numpy
import cftest

raw = numpy.ones(125, dtype=numpy.uint32)
raw[10:35] = numpy.zeros(25, dtype=numpy.uint32)

print(raw)

cftest.MarkBoundaries(raw)

print(raw)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


### 6.2.3. ctypes

In [24]:
%%writefile ctytest.cpp
#include <omp.h>

extern "C" {
    void MarkBoundaries(unsigned int *raw, int N) {
        #pragma omp parallel for
        for (int x = 0; x < N; ++x) {
            raw[x] = 3;
        }
    }
}

Overwriting ctytest.cpp


In [25]:
%%bash
g++ -fPIC -fopenmp -shared -o _ctytest.so ctytest.cpp

In [26]:
%%writefile ctytest.py
import ctypes
import numpy as np

def build_MarkBoundaries():
    dll = ctypes.CDLL('./_ctytest.so', mode=ctypes.RTLD_GLOBAL)
    func = dll.MarkBoundaries
    func.argtypes = [
        ctypes.POINTER(ctypes.c_uint),
        ctypes.c_int
    ]
    return func

def MarkBoundaries(raw):
    ctytest_sum_impl = build_MarkBoundaries()
    rawPtr = raw.ctypes.data_as(ctypes.POINTER(ctypes.c_uint))

    ctytest_sum_impl(rawPtr, raw.size)

Overwriting ctytest.py


In [27]:
import numpy
import ctytest

raw = numpy.ones(125, dtype=numpy.uint32)
raw[10:35] = numpy.zeros(25, dtype=numpy.uint32)

print(raw)

ctytest.MarkBoundaries(raw)

print(raw)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3]
